## TODOs:
* Need to apply cache to all subsets of list

In [1]:
from fastai2.text.all import *
from pigboat.text.all import *
# from fastai2.basics import *

In [2]:
source = untar_data(URLs.IMDB_SAMPLE)

In [3]:
df = pd.read_csv(source/'texts.csv')
df.head(1)

,label,text,is_valid
0,negative,"Un-bleeping-believable! Meg Ryan doesn't even look her usual pert lovable self in this, which normally makes me forgive her shallow ticky acting schtick. Hard to believe she was the producer on this dog. Plus Kevin Kline: what kind of suicide trip has his career been on? Whoosh... Banzai!!! Finally this was directed by the guy who did Big Chill? Must be a replay of Jonestown - hollywood style. Wooofff!",False


In [4]:
df['text2'] = df['text']

In [5]:
ABSTAIN,POS,NEG = 'abstain','positive','negative'

In [6]:
labeller = Labeller(ABSTAIN)
col_reader,tokenizer,numericalize = mk_transform(ColReader('text2')),Tokenizer.from_df('text'),Numericalize

In [7]:
@labeller(col_reader)
def liked(x): return POS if 'liked' in x else None
@labeller(col_reader)
def hated(x): return NEG if 'hated' in x else None

In [8]:
splits = ColSplitter()(df)

In [9]:
vocab = [ABSTAIN,POS,NEG]

In [10]:
#TODO: How to not pass ABSTAIN?
# Adds option to cache results, should also fix #1
tls = TfmdLists(df, [col_reader,tokenizer,numericalize])
tasks = tasks_labels(tls, vocab, splits=splits)

In [11]:
true_lbls = TfmdLists(df, [ColReader('label'), Categorize(vocab=vocab)], splits=splits)

In [12]:
dset_train = Datasets(tls=[tasks]).train
dset_valid = Datasets(tls=[tasks,true_lbls]).valid

In [13]:
dls = DataLoaders.from_dsets(dset_train, dset_valid, drop_last=False)

In [14]:
# TODO: Hardcoded abstain as 0, but should be a variable
def acc(pred,targ):
    mask = pred!=0
    if mask.nonzero().numel() > 0:
        return (pred[mask]==targ[mask]).float().mean()
    return 0.

In [15]:
learn = Learner(dls, MajorityLabelVoter(), loss_func=lambda *_: tensor(0.), metrics=acc)

In [16]:
learn.validate()

(#2) [0.0,0.527999997138977]